In [1]:
from podcast_ad_skipper.google_cloud import *
from podcast_ad_skipper.data_preparation import *


In [2]:
bq_client = auth_gc_bigquery()
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.test9"
# columns = '[spectrogram, labels]'

Running in local environment.
Authenticated successfully with BigQuery! ✅


In [3]:
rows = get_output_query_bigquery(bq_client, table_id, limit=2000, columns="*")

Query returned 200 results


In [4]:
def new_process_function(output):
    if output:
        spectrogram_bq, labels_bq, seconds_bq, duration_bq, podcast_name_bq = [], [], [], [], []
        for row in output:
            # Use row.field_name to access fields instead of indices
            spectrogram_bq.append(np.array(json.loads(row['spectrogram'])))
            labels_bq.append(row.get('label', None))  # Use get to handle missing fields
            seconds_bq.append(row.get('seconds', None)) if 'seconds' in row else seconds_bq.append(None)
            duration_bq.append(row.get('duration', None)) if 'duration' in row else duration_bq.append(None)
            podcast_name_bq.append(row.get('podcast_name', None)) if 'podcast_name' in row else podcast_name_bq.append(None)

        return spectrogram_bq, labels_bq, seconds_bq, duration_bq, podcast_name_bq

In [5]:
processed_columns = new_process_function(rows)

In [6]:
processed_columns

([array([[-35.97001648, -42.36822891, -62.23044205, ..., -71.00019836,
          -72.0766449 , -70.03687286],
         [-35.05725861, -41.16027832, -57.27714539, ..., -65.39038086,
          -63.70053864, -67.70195007],
         [-34.04182434, -40.37934875, -55.1485939 , ..., -58.00960159,
          -61.10978317, -70.47742462],
         ...,
         [-70.2157135 , -63.38198471, -59.55641556, ..., -80.        ,
          -80.        , -80.        ],
         [-79.83789062, -73.42992401, -69.29582214, ..., -80.        ,
          -80.        , -80.        ],
         [-80.        , -80.        , -80.        , ..., -80.        ,
          -80.        , -80.        ]]),
  array([[-35.97001648, -42.36822891, -62.23044205, ..., -71.00019836,
          -72.0766449 , -70.03687286],
         [-35.05725861, -41.16027832, -57.27714539, ..., -65.39038086,
          -63.70053864, -67.70195007],
         [-34.04182434, -40.37934875, -55.1485939 , ..., -58.00960159,
          -61.10978317, -70.47742

In [7]:
len(processed_columns[0])

200

In [8]:
nans_count = 0
spec_column = processed_columns[0]

def check_for_nans(spec_column, nans_count):
    for idx, array in enumerate(spec_column):
        if np.isnan(array).any():
                # print(f"NaNs found in array at index {idx}")
            nans_count += 1
    print(nans_count)

In [9]:
check_for_nans(spec_column, nans_count)

0


In [10]:
44/2000

0.022

In [11]:
spec_column = processed_columns[0]
label_column = processed_columns[1]

def check_for_and_delete_nans(spec_column, label_column, nans_count):
    cleaned_spec_column = []
    cleaned_label_column = []
    # New list to store non-NaN arrays
    for idx, array in enumerate(spec_column):
        if np.isnan(array).any():
            nans_count += 1
        else:
            cleaned_spec_column.append(array)
            cleaned_label_column.append(label_column[idx])

    print(f"Total NaNs found: {nans_count}")
    return cleaned_spec_column, cleaned_label_column

In [12]:
cleaned_spec_column, cleaned_label_column = check_for_and_delete_nans(spec_column, label_column, nans_count)

Total NaNs found: 0


In [13]:
len(cleaned_spec_column)

200

In [178]:
0.22*40000

8800.0

In [179]:
len(cleaned_label_column)

1956